In [5]:
from sqlalchemy import create_engine, URL
from sqlalchemy.orm import sessionmaker

In [8]:
url = URL.create(
    drivername="postgresql+psycopg2",  # driver name = postgresql + the library we are using (psycopg2)
    username='testuser',
    password='testpassword',
    host='localhost',
    database='postgres',
    port=5234
)

engine = create_engine(url, echo=True)
session_pool = sessionmaker(bind=engine)


In [14]:
from sqlalchemy import text
with session_pool() as session:
    results = session.execute(text("""
        CREATE TABLE IF NOT EXISTS users (
            telegram_id INTEGER PRIMARY KEY,
            full_name VARCHAR(255),
            username VARCHAR(255),
            language_code VARCHAR(255),
            created_at TIMESTAMP default now(),
            referrer_id BIGINT,
            FOREIGN KEY (referrer_id) 
                REFERENCES users(telegram_id)
                ON DELETE SET NULL
        );
    """))
    session.commit()

2023-06-25 16:47:52,534 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 16:47:52,535 INFO sqlalchemy.engine.Engine 
        CREATE TABLE IF NOT EXISTS users (
            telegram_id INTEGER PRIMARY KEY,
            full_name VARCHAR(255),
            username VARCHAR(255),
            language_code VARCHAR(255),
            created_at TIMESTAMP default now(),
            referrer_id BIGINT,
            FOREIGN KEY (referrer_id) 
                REFERENCES users(telegram_id)
                ON DELETE SET NULL
        );
    
2023-06-25 16:47:52,536 INFO sqlalchemy.engine.Engine [cached since 33.98s ago] {}
2023-06-25 16:47:52,542 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
from sqlalchemy import text
with session_pool() as session:
    results = session.execute(text("""
        INSERT INTO users (telegram_id, full_name, username, language_code)
        VALUES (1, 'John Doe', 'johndoe', 'en');
        
        INSERT INTO users (telegram_id, full_name, username, language_code, referrer_id)
        VALUES (2, 'Jane Doe', 'janedoe', 'en', 1);                    
    """))
    session.commit()

2023-06-25 16:49:39,875 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 16:49:39,877 INFO sqlalchemy.engine.Engine 
        INSERT INTO users (telegram_id, full_name, username, language_code)
        VALUES (1, 'John Doe', 'johndoe', 'en');
        
        INSERT INTO users (telegram_id, full_name, username, language_code, referrer_id)
        VALUES (2, 'Jane Doe', 'janedoe', 'en', 1);                    
    
2023-06-25 16:49:39,877 INFO sqlalchemy.engine.Engine [generated in 0.00062s] {}
2023-06-25 16:49:39,879 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
from sqlalchemy import text
with session_pool() as session:
    results = session.execute(text("""
        SELECT * FROM users
    """))
    for row in results:
        print(row)

2023-06-25 16:49:41,368 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 16:49:41,369 INFO sqlalchemy.engine.Engine 
        SELECT * FROM users
    
2023-06-25 16:49:41,370 INFO sqlalchemy.engine.Engine [cached since 401.4s ago] {}
(1, 'John Doe', 'johndoe', 'en', datetime.datetime(2023, 6, 25, 8, 49, 39, 878249), None)
(2, 'Jane Doe', 'janedoe', 'en', datetime.datetime(2023, 6, 25, 8, 49, 39, 878249), 1)
2023-06-25 16:49:41,371 INFO sqlalchemy.engine.Engine ROLLBACK


In [21]:
from sqlalchemy import text
with session_pool() as session:
    results = session.execute(text(f"""
        SELECT full_name, username FROM users WHERE telegram_id = :telegram_id
    """).params(telegram_id=123))
    for row in results:
        print(row)
    session.commit()

2023-06-25 17:11:21,409 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 17:11:21,410 INFO sqlalchemy.engine.Engine 
        SELECT full_name, username FROM users WHERE telegram_id = %(telegram_id)s
    
2023-06-25 17:11:21,410 INFO sqlalchemy.engine.Engine [cached since 31.9s ago] {'telegram_id': 123}
2023-06-25 17:11:21,412 INFO sqlalchemy.engine.Engine COMMIT
